<a href="https://colab.research.google.com/github/YashMaxy/Adv_Regression/blob/main/attack_workflow_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install textattack[tensorflow] --q
!pip install numpy==1.20.0 --q
!pip install transformers --q
!pip install datasets --q
!pip install sentence_transformers --q

     |████████████████████████████████| 415 kB 5.2 MB/s 
     |████████████████████████████████| 4.7 MB 68.9 MB/s 
     |████████████████████████████████| 3.6 MB 51.5 MB/s 
     |████████████████████████████████| 401 kB 54.5 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
     |████████████████████████████████| 769 kB 66.7 MB/s 
     |████████████████████████████████| 290 kB 77.9 MB/s 
     |████████████████████████████████| 41.4 MB 1.6 MB/s 
     |████████████████████████████████| 60 kB 8.1 MB/s 
     |████████████████████████████████| 454.3 MB 17 kB/s 
     |████████████████████████████████| 125 kB 73.3 MB/s 
     |████████████████████████████████| 462 kB 78.4 MB/s 
     |████████████████████████████████| 4.6 MB 44.8 MB/s 
     |████████████████████████████████| 56 kB 5.8 MB/s 
     |████████████████████████████████| 212 kB 57.3 MB/s 
     |████████████████████████████████| 141 kB 77.3 MB/s 
     |████████████████████████████████| 132 kB 58.5 MB/s 
     |█████████████

In [ ]:
config = {
    'model_name' : 'bert-base-uncased',
    'num_class' : 1,
    'dataset' : 'sid-157',
    'log_dir' : '/content/log',
    'epochs' : 20,
    'output_dir' : '/content/checkpoint',
    'batch_size' : 16,
    'warmup_steps' : 500,
    'warmup_decay' : 0.01,
    'model_dir' : '/content/model'
}

## Utils

In [ ]:
import logging
import os
from collections import Counter

import numpy as np
import pandas as pd
import transformers
from datasets import Dataset, load_dataset, load_metric
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from tqdm import tqdm
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          AutoTokenizer, EvalPrediction, HfArgumentParser,
                          PretrainedConfig, Trainer, TrainingArguments,
                          default_data_collator, set_seed)
from transformers.trainer_utils import is_main_process

import nltk
import spacy
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

nltk.download("wordnet")
import copy
import glob
from collections import Counter

import torch
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, porter
from scipy import special
from scipy.stats import pearsonr
from transformers import AdamW, EarlyStoppingCallback


def clear_punctuation(row: str) -> str:
    punctuations = "!()-[]{};:\,<>./?"
    no_punct = ""
    for char in row:
        if char not in punctuations:
            # print("not in",char)
            no_punct = no_punct + char
        elif char in punctuations:
            # print("in",char)
            no_punct += " "
    return no_punct.lower() + " "


lemmatizer = WordNetLemmatizer()
stemmer = porter.PorterStemmer()


def nltk_process_sentence(sentence):
    """remove stop words, stem and lemmatize."""
    _stem = [stemmer.stem(i) for i in sentence.split()]
    _lemma = [lemmatizer.lemmatize(i, pos="v") for i in sentence.split()]
    _stop_words = [i for i in _lemma if i not in stopwords.words("english")]

    output = " ".join(_stop_words)

    return output


class create_data:
    def __init__(
        self,
        data,
        sid,
        model_name,
        rescale=False,
        no_punct=False,
        rescale_type=0,
        prepend_prompt=False,
    ):

        # sklearn data scaler for normalizing the dataframe
        self.scaler = MinMaxScaler()
        stratify_cols = ["content_rating"]
        self.pretrained_model_name = model_name
        self.sid_data = data[data["sID"] == sid]
        self.sid_data["content_rating"] = self.sid_data["content_rating"].astype(float)
        self.prepend_prompt = prepend_prompt

        print(f"Total data points: {len(self.sid_data)}")

        # nltk modules for stemming and lemmatization
        # self.lemmatizer = WordNetLemmatizer()
        # self.stemmer = porter.PorterStemmer()

        # Rescale the data, if needed
        # If rescale_type is 1, change all values below 1 to 1.
        # If rescale_type is 4, change all values ablve 4 to 4.
        if rescale:
            # content_ratings = []
            if rescale_type == 1:
                self.sid_data[self.sid_data["content_rating"] < 1] = 1.0

            elif rescale_type == 4:
                self.sid_data[self.sid_data["content_rating"] > 4] = 4.0

            else:
                self.sid_data[self.sid_data["content_rating"] < 1] = 1.0
                self.sid_data[self.sid_data["content_rating"] > 4] = 4.0

            # self.sid_data['content_rating'] = content_ratings

        # Clear punctuation, if needed
        if no_punct:
            self.sid_data["transcript"] = self.sid_data["transcript"].apply(
                clear_punctuation
            )

        # scale the data based on the min-max value
        self.sid_data["content_rating"] = [
            float(int(x + 0.5)) for x in self.sid_data["content_rating"].tolist()
        ]
        self.scaler.fit(self.sid_data[["content_rating"]])
        self.sid_data["content_rating_scaled"] = self.scaler.transform(
            self.sid_data[["content_rating"]]
        )
        # print(self.sid_data["content_rating_scaled"])
        self.sid_data["stratify"] = self.sid_data[stratify_cols].apply(
            lambda x: "_".join(x.dropna().astype(str)), axis=1
        )
        self.tokenizer = AutoTokenizer.from_pretrained(self.pretrained_model_name)

        # self.arti_data = pd.read_excel('data/ucf_training_data_Arti.xlsx')
        # self.arti_data = self.arti_data[self.arti_data['type']=='arti']
        # self.arti_data = self.arti_data[self.arti_data['sID']==sid]
        # self.arti_data["content_rating_scaled"] = self.scaler.transform(self.arti_data[["content_rating"]])

    def nltk_preprocess(self, sentences, run=False):
        """remove stop words, stem and lemmatize."""
        # print(f"Number of sentences: {len(sentences)}")

        if run:
            output = []

            for sentence in sentences:
                _stem = [self.stemmer.stem(i) for i in sentence.split()]
                _lemma = [
                    self.lemmatizer.lemmatize(i, pos="v") for i in sentence.split()
                ]
                _stop_words = [i for i in _lemma if i not in stopwords.words("english")]

                output.append(" ".join(_stop_words))

            return output

        return sentences

    def preprocess_function(self, examples, use_bare=False):
        """
        Prepo
        """
        # Tokenize the texts
        args = examples["transcript"]
        if self.prepend_prompt:
            sid_prompt = examples["prompt"]

        if use_bare:
            args = self.nltk_preprocess(args)

        # If true, prepend the question to the response with a
        # [SEP] token in  between
        if self.prepend_prompt:
            print("Prepending the question prompt to each answer")
            result = self.tokenizer(sid_prompt, args, padding=True, truncation=True)
        else:
            result = self.tokenizer(args, padding=True, truncation=True)

        result["input_ids"] = np.array(result["input_ids"], dtype="int64")
        result["attention_mask"] = np.array(result["attention_mask"], dtype="int64")
        # result["label"] = [i for i in examples["content_rating_scaled"]]
        result["label"] = [i for i in examples["content_rating"]]
        # print(result.keys())
        return result

    def create_k_fold(self, fold=5):
        folds = StratifiedKFold(
            n_splits=fold,
            shuffle=True,
            random_state=42,
        )
        splits = dict()
        for fold, (trn, val) in enumerate(
            folds.split(self.sid_data, self.sid_data["stratify"])
        ):
            splits[fold] = dict()
            splits[fold]["train_data"] = self.sid_data.iloc[trn, :]
            splits[fold]["valid_data"] = self.sid_data.iloc[val, :]

        return splits

    def n_splits(self, fold):
        splits = dict()
        for i in range(fold):
            splits[i] = dict()

            train_set, test_set = train_test_split(
                self.sid_data,
                test_size=0.2,
                stratify=self.sid_data["stratify"],
                random_state=None,
            )
            splits[i]["train_data"] = train_set
            splits[i]["valid_data"] = test_set

        return splits

    def training_set(self):
        splits = dict()
        for i in range(1):
            splits[i] = dict()

        splits[i]["train_data"] = self.sid_data
        # For the validation set, randomly sample 100 points from the training set
        # Note: Ignore the correlation values when training the model
        splits[i]["valid_data"] = self.sid_data.sample(100)

        return splits

    def return_k_fold(self, fold, splits):
        """
        Given the fold number and the dataset split, return the
        training, testing and other dataset folds.
        """
        train_set = splits[fold]["train_data"]
        test_set = splits[fold]["valid_data"]
        test_set_bare = copy.deepcopy(test_set)
        test_set_bare["transcript"] = test_set_bare["transcript"].apply(
            lambda x: nltk_process_sentence(x)
        )

        print(Counter(train_set["content_rating"]))
        print(Counter(test_set["content_rating"]))
        print(Counter(test_set_bare["content_rating"]))

        # test_set['transcript'] = test_set['transcript'].str.replace('business risk','risk')
        # train_set,test_set = train_test_split(self.sid_data,test_size=0.2,stratify = self.sid_data["content_rating"],random_state=None)

        training_set = Dataset.from_pandas(train_set)
        testing_set = Dataset.from_pandas(test_set)
        testing_set_bare = Dataset.from_pandas(test_set_bare)
        # arti_set = Dataset.from_pandas(self.arti_data)
        columns_to_return = ["input_ids", "label", "attention_mask"]

        training_loader = training_set.map(
            self.preprocess_function, batched=True, load_from_cache_file=False
        )
        training_loader.set_format(type="torch", columns=columns_to_return)

        # Load the test dataset
        testing_loader = testing_set.map(
            self.preprocess_function, batched=True, load_from_cache_file=False
        )
        testing_loader.set_format(type="torch", columns=columns_to_return)

        # Load the test dataset with the nltk pre-processing
        testing_loader_bare = testing_set_bare.map(
            # self.preprocess_function, fn_kwargs={"use_bare": True}, batched=True,load_from_cache_file=False
            self.preprocess_function,
            batched=True,
            load_from_cache_file=False,
        )
        testing_loader_bare.set_format(type="torch", columns=columns_to_return)

        # arti_loader = arti_set.map(self.preprocess_function, batched=True,load_from_cache_file=False)
        # columns_to_return = ['input_ids', 'label', 'attention_mask']
        # arti_loader.set_format(type='torch', columns=columns_to_return)

        # return training_loader,testing_loader,test_set,train_set, arti_loader, self.arti_data
        return (
            training_loader,
            testing_loader,
            testing_loader_bare,
            train_set,
            test_set,
            test_set_bare,
        )

    def rescale_data(self, original):
        return original
        # return self.scaler.inverse_transform(original)


def correlation(logits, labels):
    # class_ids =torch.tensor( [0,1,2,3,4])
    # preds = torch.nn.functional.softmax(torch.tensor(logits), dim=1)
    # predicted_values = torch.mul(preds, class_ids).sum(axis=1)
    return pearsonr(logits, labels)[0]


def dump_correlation(pred, true_l):

    output = pearsonr(pred, true_l)[0]
    # print(pred)
    return output


def dump_result(output_folder, model_name, sids, path):
    for sid in [sids]:
        result_data = pd.DataFrame()
        print(sid)
        domain_adapt = []
        distil_lst = []
        for fold in range(5):
            # epochs_39 = pd.read_csv("epochs_39_output/"+ "output" + "_epochs_39"+"_fold_"+str(fold)+ "_sid_"+str(sid)+".csv")
            distill = pd.read_csv(
                path
                + "/test_output"
                + "_"
                + model_name.split("/")[0]
                + "_fold_"
                + str(fold)
                + "_sid_"
                + str(sid)
                + ".csv"
            )
            result_data = result_data.append(distill)
            distil_lst.append(
                dump_correlation(
                    distill["predictions"].tolist(),
                    distill["content_rating_scaled"].tolist(),
                )
            )
            # domain_adapt.append(dump_correlation(epochs_39['predictions'].tolist(),epochs_39['content_rating'].tolist()))
        print(model_name, sum(distil_lst) / len(distil_lst), distil_lst)
        if model_name == "distilbert-base-uncased":
            result_data.to_csv(
                "predictions/distillbert/" + str(sid) + ".csv", index=False
            )
        else:
            result_data.to_csv(
                "predictions/domainAdapt/" + str(sid) + ".csv", index=False
            )
    return sum(distil_lst) / len(distil_lst), result_data


def get_pwl_score(score, thres):

    try:
        co = 1
        threshold_list = thres
        levels_pwl = len(threshold_list) - 2
        level_scores = []
        level_scores.append(0)

        for i in range(0, levels_pwl + 1):
            previous_level_score = level_scores[-1]
            if i == 0 or i == (levels_pwl):
                added_component = 100.0 / (levels_pwl * 2)
            else:
                added_component = 100.0 / (levels_pwl)
            new_level_score = previous_level_score + added_component
            level_scores.append(new_level_score)

        # min score is 5
        # max score is 95
        level_scores[0] = 5.0
        level_scores[-1] = 95.0
        if co == 1:
            # print("level_scores",level_scores)
            # print("level_pwl",levels_pwl)
            co = co + 1

        pwl_score = -1
        pwl_disc_score = -1
        # print(threshold_list)

        for i in range(1, levels_pwl + 2):
            if score <= threshold_list[i]:
                pwl_score = level_scores[i - 1] + (score - threshold_list[i - 1]) * (
                    level_scores[i] - level_scores[i - 1]
                ) / (threshold_list[i] - threshold_list[i - 1])
                pwl_disc_score = i - 1
                break

        if pwl_score == -1:
            pwl_score = 95
            pwl_disc_score = levels_pwl

        pwl_score = min(95, pwl_score)
        pwl_score = max(5, pwl_score)
        return pwl_score
    except:
        return 0


def get_thresholds(predicted_scores_train, train_scores):

    all_train_scores_disc = [int(score + 0.5) for score in train_scores]

    all_train_scores_sort_disc = copy.deepcopy(all_train_scores_disc)
    predicted_scores_train_sort = copy.deepcopy(predicted_scores_train)
    predicted_scores_train_dis = copy.deepcopy(predicted_scores_train)

    # sort all discrete scores
    all_train_scores_sort_disc.sort()
    predicted_scores_train_sort.sort()

    # counter will give frequency of each label.
    counts = Counter(all_train_scores_sort_disc)
    print("Counts : ", counts)

    # using counter, get index of all label. As all is sorted.
    l_0_idx = counts[0]
    l_1_idx = counts[1] + counts[0]
    l_2_idx = counts[2] + counts[1] + counts[0]
    l_3_idx = counts[3] + counts[2] + counts[1] + counts[0]
    l_4_idx = counts[4] + counts[3] + counts[2] + counts[1] + counts[0]
    l_5_idx = counts[5] + counts[4] + counts[3] + counts[2] + counts[1] + counts[0]

    # print("index : ",[l_0_idx,l_1_idx,l_2_idx,l_3_idx,l_4_idx,l_5_idx])

    # assign threshold based on the discrete sorted scores and actual label counter
    l_m = predicted_scores_train_sort[0]
    l_0 = predicted_scores_train_sort[l_0_idx - 1]
    if l_0_idx == 0:
        l_0 = 0.05
    l_1 = predicted_scores_train_sort[l_1_idx - 1]
    l_2 = predicted_scores_train_sort[l_2_idx - 1]
    l_3 = predicted_scores_train_sort[l_3_idx - 1]
    l_4 = predicted_scores_train_sort[l_4_idx - 1]
    l_5 = predicted_scores_train_sort[l_5_idx - 1]

    # return [l_m, l_0, l_1, l_2, l_3, l_4, l_5]
    return [l_0, l_1, l_2, l_3, l_4, l_5]


def float_to_buckets(predicted_labels, thresholds):
    bucketed_labels = []
    for label in predicted_labels:
        if label < thresholds[0]:
            bucketed_labels.append(0)
        elif label < thresholds[1]:
            bucketed_labels.append(1)
        elif label < thresholds[2]:
            bucketed_labels.append(2)
        elif label < thresholds[3]:
            bucketed_labels.append(3)
        elif label < thresholds[4]:
            bucketed_labels.append(4)
        else:
            bucketed_labels.append(5)

    return bucketed_labels


def make_directory(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)


def combine_dfs(folder):
    csv_files = glob.glob(os.path.join(folder, "distilbert*.csv"))
    df = []
    for filename in csv_files:
        df.append(pd.read_csv(filename))

    df = pd.concat(df)

    df.to_csv(os.path.join(folder, "combined.csv"))


[nltk_data] Downloading package wordnet to /root/nltk_data...


## Training

In [ ]:
import os
import shutil
from collections import Counter

import nltk
import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.utils.class_weight import compute_class_weight
from transformers import (AutoConfig, AutoModelForSequenceClassification,
                          EarlyStoppingCallback, Trainer, TrainingArguments)
import torch.nn as nn

# from utils import *

nltk.download("wordnet")
nltk.download('omw-1.4')

siod_data = {
    "197": pd.read_excel(
        "/content/sID 197 Final Ratings (432+50) 10 May 2022.xlsx",
        engine="openpyxl",
    ),
    "763": pd.read_excel(
        "/content/sID 763 Final Rating (107+426) Shared with AI team.xlsx",
        engine="openpyxl",
    ),
    "481": pd.read_excel(
        "/content/sID 481 Final Calibration Sheet (56+441 audios).xlsx",
        engine="openpyxl",
    ),
    # "all": pd.read_excel("./data/ucf_data_sampleID.xlsx", engine="openpyxl"),
}

prompts = {
    "197": "You are responsible for selecting and purchasing equipment to help make your company's manufacturing process more efficient. There are several well-known brands with good reputations available, along with a number of lesser known brands. How would you decide which equipment to purchase? What general factors would you consider and how would each impact your decision?",
    "763": "You are helping an event planner with an important event that involves coordinating several teams working together to setup the hired equipment, deliver the event, and tidy up afterwards. To ensure the event runs smoothly, you have been asked to prepare for anything that could go wrong. What types of problems, both internal and external to your team, could come up that would affect how well the event goes? List as many specific factors as you can.",
    "481": "You have noticed a costly mistake that affected an important customer and quickly realize that the mistake was the result of a decision you made, but no one else knows this. You can fix the mistake without anyone noticing but this would mean that antoher client might be impacted in the future. There would be no way that anyone else to identify you made the mistake.  How would you handle this situation?",
}

# Read the data
def read_and_clean_dataframe(sid):

    data = siod_data[str(sid)]
    print(data.columns)

    prompt = "blank"
    print(len(prompt.split()))
    
    data.rename(
        columns={"Transcripts": "transcript", "Final Ratings": "content_rating"},
        inplace=True,
    )

    data["sID"] = sid
    # data = data[["sampleId", "transcript", "content_rating", "sID"]]
    data["prompt"] = prompts[str(sid)]
    data = data[["sampleId", "transcript", "content_rating", "sID", "prompt"]]

    print(data.columns)

    data = data[data["transcript"] != 0]
    data = data[data["content_rating"] != -1]
    data.groupby("sID").count()
    data = data[data["content_rating"].notna()]

    print(data.transcript.iloc[0])

    return data


# Training loop

training_config = {
    "stage": 2,
    # "training_ids": [197, 481, 763],
    "training_ids": [763],
    # "training_ids": [163, 171, 7107165, 7107167, 615],
    "finetune": True,
    "folds": 5,
    "split_n": 1,
    "lr": 6e-3,
    "use_class_weighting": True,
    "a_min": 0.0,
    "a_max": 3.0,
}

assert training_config["stage"] in [1, 2, 3], "Stage has to be 1, 2 or 3!"


model_name = "distilbert-base-uncased"
output_folder = model_name.replace("-", "_")
make_directory(output_folder)

for sid in training_config["training_ids"]:

    result = {}
    lst = []
    correlation_values = []

    data = read_and_clean_dataframe(sid)

    train_path = os.path.join(output_folder, str(sid), "train")
    test_path = os.path.join(output_folder, str(sid), "test")
    test_path_bare = os.path.join(output_folder, str(sid), "test_bare")

    make_directory(train_path)
    make_directory(test_path)
    make_directory(test_path_bare)

    # Create the data class
    class_data = create_data(
        data=data,
        sid=sid,
        model_name=model_name,
        rescale=False,
        no_punct=False,
        rescale_type=6,
        prepend_prompt=False,
    )

    if training_config["stage"] == 1:
        splits = class_data.create_k_fold(training_config["folds"])
        iterations = training_config["folds"]

    elif training_config["stage"] == 2:
        splits = class_data.n_splits(training_config["split_n"])
        iterations = training_config["split_n"]

    elif training_config["stage"] == 3:
        splits = class_data.training_set()
        iterations = 1

    for fold in range(iterations):

        config = AutoConfig.from_pretrained(model_name, num_labels=1)
        model = AutoModelForSequenceClassification.from_pretrained(
            model_name, config=config
        )

        (
            training_loader,
            testing_loader,
            testing_loader_bare,
            train_set,
            test_set,
            test_set_bare,
        ) = class_data.return_k_fold(fold, splits)

        print(len(training_loader), len(testing_loader), len(testing_loader_bare))

        classes = [0,1,2,3,4,5]
        print(f"classes: {classes}")

        for param in model.base_model.parameters():
            if training_config["finetune"]:
                param.requires_grad = True
                epochs = 25
            else:
                param.requires_grad = False
                epochs = 18

        def compute_metrics(pred):
            global lst
            labels = pred.label_ids
            preds = pred.predictions
            labels = np.squeeze(labels)
            preds = np.squeeze(preds)
            corr1, _ = pearsonr(labels, preds)
            lst.append(corr1)
            return {"corr": corr1}

        if training_config["stage"] == 3:
            output_dir = os.path.join(output_folder, str(sid), "checkpoint")
        else:
            output_dir = "checkpoint"

        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,  # total number of training epochs
            per_device_train_batch_size=8,  # batch size per device during training
            logging_dir="log/",  # directory for storing logs
            logging_steps=1000,
            per_device_eval_batch_size=8,
            evaluation_strategy="epoch",
            warmup_steps=500,  # number of warmup steps for learning rate scheduler
            weight_decay=0.01,
            save_strategy="epoch",
            do_eval=True,
            metric_for_best_model="eval_loss",
            load_best_model_at_end=True,
            greater_is_better=False,
        )

        if training_config["use_class_weighting"]:
            weights = compute_class_weight(
                class_weight='balanced',
                classes=np.unique(train_set['content_rating'].tolist()),
                y=train_set['content_rating'].tolist()
            )
            unique_l = np.unique(train_set['content_rating'].tolist())
            print(f"unique_l: {unique_l}")
            print(f"old weights: {weights}")
            weights = np.clip(weights, a_min=training_config["a_min"], a_max=training_config["a_max"])
            print(f"new weights: {weights}")

        else:
            weights = [1,1,1,1,1,1]
        
        # reduce weight of the "0" class
        # weights[0] = 1.0
            
        class CustomTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False):
                global weights
                global classes
                #global unique_l
                labels = inputs.get("labels")
                # forward pass
                outputs = model(**inputs)
                logits = outputs.get("logits")
                #print("output")
                # compute custom loss (suppose one has 3 labels with different weights)
                loss_fct = nn.MSELoss()
                model_logits = logits.view(-1).cuda()
                model_labels = labels.view(-1).cuda()
                #loss = loss_fct(model_logits, model_labels)
                #return (loss, outputs) if return_outputs else loss 
            
                global_loss = 0
                
                weightDict = {}
                for i,value in zip(classes,weights):
                        weightDict[i] = value
                        
                #print("WeightedLoss :",weightDict)
                #for i in [1,2,3,4]:
                for i in classes:
                    #print(i)
                    indexes = (model_labels == i).nonzero(as_tuple=True)[0].cuda()
                    #print("ind",indexes)
                    logits_for_this_rating = torch.index_select(model_logits, 0, indexes).cuda()
                    labels_for_this_rating = torch.index_select(model_labels, 0, indexes).cuda()
                    losses = loss_fct(logits_for_this_rating, labels_for_this_rating)
                    
                    #print("Loss:",losses)
                    if(torch.isnan(losses)):
                        global_loss+= 0
                        continue
                    global_loss+= weightDict[i]*losses
                    #print("Weighted Loss",weightDict[i]*loss)
                    #global_loss += loss
                    #print("Loss :",loss)
                    #print("wt :",weightDict[i.item()]*loss)
                    #print("loss :",i,":",weightDict[i.item()]*loss,(global_loss))
                    #print(global_loss)

                return (global_loss, outputs) if return_outputs else global_loss


        trainer = Trainer(
        # trainer = CustomTrainer(
            model=model,
            args=training_args,
            train_dataset=training_loader,
            eval_dataset=testing_loader,
            compute_metrics=compute_metrics,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
            data_collator=None,
        )
        trainer.train()

        # Generate predictions for the respective sets
        outs = trainer.predict(testing_loader)
        outs_bare = trainer.predict(testing_loader_bare)
        outs_train = trainer.predict(training_loader)

        test_set["predictions"] = [i[0] for i in outs.predictions]
        test_set["predictions_rescaled"] = class_data.rescale_data(
            test_set[["predictions"]]
        )

        train_set["predictions"] = [i[0] for i in outs_train.predictions]
        train_set["predictions_rescaled"] = class_data.rescale_data(
            train_set[["predictions"]]
        )

        test_set_bare["predictions"] = [i[0] for i in outs_bare.predictions]
        test_set_bare["predictions_rescaled"] = class_data.rescale_data(
            test_set_bare[["predictions"]]
        )

        # Append the correlation values per iteration to a dataframe
        if training_config["stage"] == 2:
            correlation_values.append(
                {
                    "iteration": fold,
                    "test_corr": pearsonr(
                        test_set["content_rating"], test_set["predictions"]
                    )[0],
                    "train_corr": pearsonr(
                        train_set["content_rating"], train_set["predictions"]
                    )[0],
                }
            )

        thresholds = get_thresholds(
            train_set["predictions_rescaled"].tolist(),
            train_set["content_rating"].tolist(),
        )
        print(f"thresholds: {thresholds}")

        def append_pwl_scores(output_dict, thresholds):
            pwl = []
            for score in output_dict["predictions_rescaled"].tolist():
                pwl.append(get_pwl_score(score, thresholds))
            output_dict["pwl_score_100"] = pwl
            output_dict["actual_pwl_score_100"] = output_dict["content_rating"] * 20

            return output_dict

        test_set = append_pwl_scores(test_set, thresholds)
        test_set_bare = append_pwl_scores(test_set_bare, thresholds)

        # Note: Append a score of 0 for the thresholds list when
        # processing the training set
        thresholds[0] = 0
        train_set = append_pwl_scores(train_set, thresholds)

        # Save csv files
        if training_config["stage"] == 1:
            test_set.to_csv(
                os.path.join(
                    test_path,
                    f"{model_name.split('/')[0]}_fold_{str(fold)}_sid_{str(sid)}.csv",
                )
            )
            test_set_bare.to_csv(
                os.path.join(
                    test_path_bare,
                    f"{model_name.split('/')[0]}_fold_{str(fold)}_sid_{str(sid)}.csv",
                )
            )
            train_set.to_csv(
                os.path.join(
                    train_path,
                    f"{model_name.split('/')[0]}_fold_{str(fold)}_sid_{str(sid)}.csv",
                )
            )
        
        print(f"Epochs: {sid}")

        if training_config["stage"] == 3:
            trainer.save_model(output_dir)

        # Delete all checkpoints if the model is not in the final stage
        print("Removing checkpoint directory")
        shutil.rmtree("checkpoint")

    if training_config["stage"] == 2:
        correlation_values = pd.DataFrame(correlation_values)
        correlation_values.to_csv(
            os.path.join(output_folder, str(sid), "correlation_values.csv"),
            index=False,
        )


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Index(['sampleId', 'Transcripts', 'Rahul Final Ratings',
       'Vivaksha _Final Ratings', 'Shloka_Final Ratings', 'Final Ratings',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10'],
      dtype='object')
1
Index(['sampleId', 'transcript', 'content_rating', 'sID', 'prompt'], dtype='object')
I'm so I'm going to help the event planner and I'm going to list the things that could go wrong internally and externally. And so manly, and I think that what could go wrong is that the Hyder equipment doesn't come in time, and this happens a lot of events and things don't turn up and things are alert and things just go missing, so that could go wrong. You know, companies aren't very reliable sometimes and then also with equipment as well being electronics mainly and they couldn't work. They might not work, so we need maybe backups and things like lights and that could go wrong. People could drop them, they could smash anything like that and also with the tidying up after

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.12.2",
  "vocab_size": 30522
}

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebf

Counter({3.0: 173, 1.0: 78, 4.0: 68, 2.0: 55, 5.0: 25, 0.0: 10})
Counter({3.0: 44, 1.0: 19, 4.0: 17, 2.0: 14, 5.0: 6, 0.0: 3})
Counter({3.0: 44, 1.0: 19, 4.0: 17, 2.0: 14, 5.0: 6, 0.0: 3})


  0%|          | 0/1 [00:00<?, ?ba/s]

...

  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1300


Epoch,Training Loss,Validation Loss,Corr
1,No log,3.684642,0.117830
2,No log,1.071562,0.646161
3,No log,0.668096,0.741702
4,No log,0.770779,0.714354
5,No log,0.681474,0.786441
6,No log,0.929545,0.730308
7,No log,0.851387,0.759027
8,No log,0.778035,0.696823


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sampleId, prompt, sID, __index_level_0__, content_rating, stratify, transcript, content_rating_scaled.
***** Running Evaluation *****
  Num examples = 103
  Batch size = 8
Saving model checkpoint to checkpoint/checkpoint-52
Configuration saved in checkpoint/checkpoint-52/config.json
Model weights saved in checkpoint/checkpoint-52/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sampleId, prompt, sID, __index_level_0__, content_rating, stratify, transcript, content_rating_scaled.
***** Running Evaluation *****
  Num examples = 103
  Batch size = 8
Saving model checkpoint to checkpoint/checkpoint-104
Configuration saved in checkpoint/checkpoint-104/config.json
Model weights saved in checkpoint/checkpoint-104/pytorch_m

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sampleId, prompt, sID, __index_level_0__, content_rating, stratify, transcript, content_rating_scaled.
***** Running Prediction *****
  Num examples = 103
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sampleId, prompt, sID, __index_level_0__, content_rating, stratify, transcript, content_rating_scaled.
***** Running Prediction *****
  Num examples = 409
  Batch size = 8


Counts :  Counter({3: 173, 1: 78, 4: 68, 2: 55, 5: 25, 0: 10})
thresholds: [0.8160083293914795, 1.57112717628479, 2.4066436290740967, 3.366511821746826, 3.492882013320923, 3.5841774940490723]
Epochs: 763
Removing checkpoint directory


In [ ]:
shutil.rmtree("/content/distilbert_base_uncased/763/checkpoint")

In [ ]:
test_set.head(3)

,sampleId,transcript,content_rating,sID,prompt,content_rating_scaled,stratify,predictions,predictions_rescaled,pwl_score_100,actual_pwl_score_100
332,5e89ec793c01b3685a00bb12_7700763.wav,So there are several things that could be cons...,3.0,763,You are helping an event planner with an impor...,0.6,3.0,1.979452,1.979452,24.717734,60.0
5,5ae96d1878b0060001e69b8a_7700763.wav,"There are many things that can go wrong, proba...",3.0,763,You are helping an event planner with an impor...,0.6,3.0,1.781934,1.781934,18.807667,60.0
511,6136e715cf2e36f0dfb894e5_7700763.wav,We need to focus firstly on delegating tasks a...,1.0,763,You are helping an event planner with an impor...,0.2,1.0,0.904262,0.904262,5.876552,20.0


In [ ]:
trainer = Trainer(
# trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=training_loader,
    eval_dataset=testing_loader,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
    data_collator=None,
)

print(trainer.evaluate())

trainer.save_model('/content/distilbert_base_uncased/763')

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sampleId, prompt, sID, __index_level_0__, content_rating, stratify, transcript, content_rating_scaled.
***** Running Evaluation *****
  Num examples = 103
  Batch size = 8


Saving model checkpoint to /content/distilbert_base_uncased/763
Configuration saved in /content/distilbert_base_uncased/763/config.json


{'eval_loss': 0.6680962443351746, 'eval_corr': 0.7417024276123362, 'eval_runtime': 1.3237, 'eval_samples_per_second': 77.812, 'eval_steps_per_second': 9.821}


Model weights saved in /content/distilbert_base_uncased/763/pytorch_model.bin


In [ ]:
test_set.to_csv('/content/distilbert_base_uncased/763/test/test_dump.csv')
test_set_bare.to_csv('/content/distilbert_base_uncased/763/test_bare/test_set_bare_dump.csv')
train_set.to_csv('/content/distilbert_base_uncased/763/train/train_dump.csv')

## Attack

In [ ]:
siod_data['197'][['Transcripts', 'Final Ratings']].values[0]

array(["I'd look at all the reviews. To be perfectly honest with you, I would also look for recommendations and I probably go into a store as well just to have like I can get a good feel about stuff. But yeah, I had. Recommendations I think. ",
       2], dtype=object)

In [ ]:
model_file = '/content/drive/MyDrive/Attack_SIOD/763/pytorch_model.bin'
config = AutoConfig.from_pretrained(model_name, num_labels=1)
model = AutoModelForSequenceClassification.from_pretrained(
    model_file, config=config
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from textattack import Attack
from textattack.models.wrappers import PyTorchModelWrapper, HuggingFaceModelWrapper
from textattack.search_methods import GreedySearch, GreedyWordSwapWIR
from textattack.goal_functions import UntargetedClassification, ClassificationGoalFunction, GoalFunction
from textattack.transformations import WordSwapEmbedding
from textattack.transformations import (
    CompositeTransformation,
    WordInsertionMaskedLM,
    WordMergeMaskedLM,
    WordSwapMaskedLM,
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder, BERT
from textattack.constraints.grammaticality import PartOfSpeech
from textattack.constraints.pre_transformation import (
    InputColumnModification,
    MaxModificationRate,
    RepeatModification,
    StopwordModification,
)
from textattack.constraints.semantics import WordEmbeddingDistance

from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Goal function

In [ ]:
""".. _goal_function:

GoalFunction Class
===========================================================
"""


from abc import ABC, abstractmethod

import lru
import numpy as np
import torch

from textattack.goal_function_results.goal_function_result import (
    GoalFunctionResultStatus,
)
from textattack.shared import validators
from textattack.shared.utils import ReprMixin


class GoalFunction(ReprMixin, ABC):
    """Evaluates how well a perturbed attacked_text object is achieving a
    specified goal.

    Args:
        model_wrapper (:class:`~textattack.models.wrappers.ModelWrapper`):
            The victim model to attack.
        maximizable(:obj:`bool`, `optional`, defaults to :obj:`False`):
            Whether the goal function is maximizable, as opposed to a boolean result of success or failure.
        query_budget (:obj:`float`, `optional`, defaults to :obj:`float("in")`):
            The maximum number of model queries allowed.
        model_cache_size (:obj:`int`, `optional`, defaults to :obj:`2**20`):
            The maximum number of items to keep in the model results cache at once.
    """

    def __init__(
        self,
        model_wrapper,
        maximizable=False,
        use_cache=True,
        query_budget=float("inf"),
        model_batch_size=32,
        model_cache_size=2**20,
    ):
        validators.validate_model_goal_function_compatibility(
            self.__class__, model_wrapper.model.__class__
        )
        self.model = model_wrapper
        self.maximizable = maximizable
        self.use_cache = use_cache
        self.use_cache = False
        self.query_budget = query_budget
        self.batch_size = model_batch_size
        if self.use_cache:
            self._call_model_cache = lru.LRU(model_cache_size)
        else:
            self._call_model_cache = None


    def clear_cache(self):
        if self.use_cache:
            self._call_model_cache.clear()

    def init_attack_example(self, attacked_text, ground_truth_output):
        """Called before attacking ``attacked_text`` to 'reset' the goal
        function and set properties for this example."""
        self.initial_attacked_text = attacked_text
        self.ground_truth_output = ground_truth_output
        self.num_queries = 0
        result, _ = self.get_result(attacked_text, check_skip=True)
        return result, _

    def get_output(self, attacked_text):
        """Returns output for display based on the result of calling the
        model."""
        return self._get_displayed_output(self._call_model([attacked_text])[0])

    def get_result(self, attacked_text, **kwargs):
        """A helper method that queries ``self.get_results`` with a single
        ``AttackedText`` object."""
        results, search_over = self.get_results([attacked_text], **kwargs)
        result = results[0] if len(results) else None
        return result, search_over

    def get_results(self, attacked_text_list, check_skip=False):
        """For each attacked_text object in attacked_text_list, returns a
        result consisting of whether or not the goal has been achieved, the
        output for display purposes, and a score.

        Additionally returns whether the search is over due to the query
        budget.
        """
        results = []
        if self.query_budget < float("inf"):
            queries_left = self.query_budget - self.num_queries
            attacked_text_list = attacked_text_list[:queries_left]
        self.num_queries += len(attacked_text_list)
        model_outputs = self._call_model(attacked_text_list)
        # print(f'model_outputs = {model_outputs}')
        for attacked_text, raw_output in zip(attacked_text_list, model_outputs):

            displayed_output = self._get_displayed_output(raw_output)
            # print(f'raw_output = {raw_output} vs displayed_output = {displayed_output}')
            goal_status = self._get_goal_status(
                raw_output, attacked_text, check_skip=check_skip
            )
            goal_function_score = self._get_score(raw_output, attacked_text)
            results.append(
                self._goal_function_result_type()(
                    attacked_text,
                    raw_output,
                    displayed_output,
                    goal_status,
                    goal_function_score,
                    self.num_queries,
                    self.ground_truth_output,
                )
            )
        # print(f'gaol-res:{results}')
        return results, self.num_queries == self.query_budget

    def _get_goal_status(self, model_output, attacked_text, check_skip=False):
        should_skip = check_skip and self._should_skip(model_output, attacked_text)
        if should_skip:
            return GoalFunctionResultStatus.SKIPPED
        if self.maximizable:
            return GoalFunctionResultStatus.MAXIMIZING
        if self._is_goal_complete(model_output, attacked_text):
            return GoalFunctionResultStatus.SUCCEEDED
        return GoalFunctionResultStatus.SEARCHING

    @abstractmethod
    def _is_goal_complete(self, model_output, attacked_text):
        raise NotImplementedError()

    def _should_skip(self, model_output, attacked_text):
        return self._is_goal_complete(model_output, attacked_text)

    @abstractmethod
    def _get_score(self, model_output, attacked_text):
        raise NotImplementedError()

    def _get_displayed_output(self, raw_output):
        return raw_output

    @abstractmethod
    def _goal_function_result_type(self):
        """Returns the class of this goal function's results."""
        raise NotImplementedError()

    @abstractmethod
    def _process_model_outputs(self, inputs, outputs):
        """Processes and validates a list of model outputs.

        This is a task-dependent operation. For example, classification
        outputs need to make sure they have a softmax applied.
        """
        raise NotImplementedError()

    def _call_model_uncached(self, attacked_text_list):
        """Queries model and returns outputs for a list of AttackedText
        objects."""
        if not len(attacked_text_list):
            return []

        inputs = [at.tokenizer_input for at in attacked_text_list]
        outputs = []
        i = 0
        while i < len(inputs):
            batch = inputs[i : i + self.batch_size]
            batch_preds = self.model(batch)

            # Some seq-to-seq models will return a single string as a prediction
            # for a single-string list. Wrap these in a list.
            if isinstance(batch_preds, str):
                batch_preds = [batch_preds]

            # Get PyTorch tensors off of other devices.
            if isinstance(batch_preds, torch.Tensor):
                batch_preds = batch_preds.cpu()

            if isinstance(batch_preds, list):
                outputs.extend(batch_preds)
            elif isinstance(batch_preds, np.ndarray):
                outputs.append(torch.tensor(batch_preds))
            else:
                outputs.append(batch_preds)
            i += self.batch_size

        if isinstance(outputs[0], torch.Tensor):
            outputs = torch.cat(outputs, dim=0)

        assert len(inputs) == len(
            outputs
        ), f"Got {len(outputs)} outputs for {len(inputs)} inputs"
        # print(f'gaol fun-out : {outputs}')

        return self._process_model_outputs(attacked_text_list, outputs)

    def _call_model(self, attacked_text_list):
        """Gets predictions for a list of ``AttackedText`` objects.

        Gets prediction from cache if possible. If prediction is not in
        the cache, queries model and stores prediction in cache.
        """
        if not self.use_cache:
            return self._call_model_uncached(attacked_text_list)
        else:
            uncached_list = []
            for text in attacked_text_list:
                if text in self._call_model_cache:
                    # Re-write value in cache. This moves the key to the top of the
                    # LRU cache and prevents the unlikely event that the text
                    # is overwritten when we store the inputs from `uncached_list`.
                    self._call_model_cache[text] = self._call_model_cache[text]
                else:
                    uncached_list.append(text)
            uncached_list = [
                text
                for text in attacked_text_list
                if text not in self._call_model_cache
            ]
            outputs = self._call_model_uncached(uncached_list)
            for text, output in zip(uncached_list, outputs):
                self._call_model_cache[text] = output
            all_outputs = [self._call_model_cache[text] for text in attacked_text_list]
            all_outputs = self._call_model_uncached(attacked_text_list)
            # print(f'all_outputs : {all_outputs}')
            return all_outputs

    def extra_repr_keys(self):
        attrs = []
        if self.query_budget < float("inf"):
            attrs.append("query_budget")
        if self.maximizable:
            attrs.append("maximizable")
        return attrs

    def __getstate__(self):
        state = self.__dict__.copy()
        if self.use_cache:
            state["_call_model_cache"] = self._call_model_cache.get_size()
        return state

    def __setstate__(self, state):
        self.__dict__ = state
        if self.use_cache:
            self._call_model_cache = lru.LRU(state["_call_model_cache"])


In [ ]:
from textattack.goal_function_results import ClassificationGoalFunctionResult
class RegressionGoalFunction(GoalFunction):
    """A goal function defined on a model that outputs a probability for some
    number of classes."""

    def _process_model_outputs(self, inputs, scores):
        """Processes and validates a list of model outputs.

        This is a task-dependent operation. For example, classification
        outputs need to have a softmax applied.
        """
        # Automatically cast a list or ndarray of predictions to a tensor.
        # print(f'process model output input= {scores}')
        if isinstance(scores, list) or isinstance(scores, np.ndarray):
            scores = torch.tensor(scores)

        # Ensure the returned value is now a tensor.
        if not isinstance(scores, torch.Tensor):
            raise TypeError(
                "Must have list, np.ndarray, or torch.Tensor of "
                f"scores. Got type {type(scores)}"
            )

        # Validation check on model score dimensions
        if scores.ndim == 1:
            # Unsqueeze prediction, if it's been squeezed by the model.
            if len(inputs) == 1:
                scores = scores.unsqueeze(dim=0)
            else:
                raise ValueError(
                    f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
                )
        elif scores.ndim != 2:
            # If model somehow returns too may dimensions, throw an error.
            raise ValueError(
                f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
            )
        elif scores.shape[0] != len(inputs):
            # If model returns an incorrect number of scores, throw an error.
            raise ValueError(
                f"Model return score of shape {scores.shape} for {len(inputs)} inputs."
            )
        # elif not ((scores.sum(dim=1) - 1).abs() < 1e-6).all():
        #     # Values in each row should sum up to 1. The model should return a
        #     # set of numbers corresponding to probabilities, which should add
        #     # up to 1. Since they are `torch.float` values, allow a small
        #     # error in the summation.
        #     if scores.numel()>1:
        #         scores = torch.nn.functional.softmax(scores, dim=1)
        #         print('applying softmax')
        #         if not ((scores.sum(dim=1) - 1).abs() < 1e-6).all():
        #             raise ValueError("Model scores do not add up to 1.")
        # print(f'process model output output= {scores}')
        return scores.cpu().float()

    def _goal_function_result_type(self):
        """Returns the class of this goal function's results."""
        return ClassificationGoalFunctionResult

    def extra_repr_keys(self):
        return []


    def _get_displayed_output(self, raw_output):
        return raw_output#int(raw_output.argmax())

In [ ]:
# from pandas._libs.algos import groupsort_indexer
class TargetedRegression(RegressionGoalFunction):
    """An untargeted attack on classification models which attempts to minimize
    the score of the correct label until it is no longer the predicted label.

    Args:
        target_max_score (float): If set, goal is to reduce model output to
            below this score. Otherwise, goal is to change the overall predicted
            class.
    """

    def __init__(self, *args, target_max_score=None, **kwargs):
        self.target_max_score = target_max_score
        super().__init__(*args, **kwargs)

    def _is_goal_complete(self, model_output, _):
        if self.target_max_score:
            return model_output[self.ground_truth_output] < self.target_max_score
        elif (model_output.numel() == 1):
            # print(self.ground_truth_output, model_output.item())
            # print(self.cda)
            return model_output.item() - self.ground_truth_output  >= 1.5
        else:
            return model_output.argmax() != self.ground_truth_output

    def _get_score(self, model_output, _):
        # If the model outputs a single number and the ground truth output is
        # a float, we assume that this is a regression task.
        if (model_output.numel() == 1) :
            # print(model_output.item(), self.ground_truth_output)
            return model_output.item() - self.ground_truth_output 
        else:
            return 1 - model_output[self.ground_truth_output]

## Attack.py

In [ ]:
"""
Attack Class
============
"""

from collections import OrderedDict
from typing import List, Union

import lru
import torch

import textattack
from textattack.attack_results import (
    FailedAttackResult,
    MaximizedAttackResult,
    SkippedAttackResult,
    SuccessfulAttackResult,
)
from textattack.constraints import Constraint, PreTransformationConstraint
from textattack.goal_function_results import GoalFunctionResultStatus
# from textattack.goal_functions import GoalFunction
from textattack.models.wrappers import ModelWrapper
from textattack.search_methods import SearchMethod
from textattack.shared import AttackedText, utils
from textattack.transformations import CompositeTransformation, Transformation


class Attack:
    """An attack generates adversarial examples on text.

    An attack is comprised of a goal function, constraints, transformation, and a search method. Use :meth:`attack` method to attack one sample at a time.

    Args:
        goal_function (:class:`~textattack.goal_functions.GoalFunction`):
            A function for determining how well a perturbation is doing at achieving the attack's goal.
        constraints (list of :class:`~textattack.constraints.Constraint` or :class:`~textattack.constraints.PreTransformationConstraint`):
            A list of constraints to add to the attack, defining which perturbations are valid.
        transformation (:class:`~textattack.transformations.Transformation`):
            The transformation applied at each step of the attack.
        search_method (:class:`~textattack.search_methods.SearchMethod`):
            The method for exploring the search space of possible perturbations
        transformation_cache_size (:obj:`int`, `optional`, defaults to :obj:`2**15`):
            The number of items to keep in the transformations cache
        constraint_cache_size (:obj:`int`, `optional`, defaults to :obj:`2**15`):
            The number of items to keep in the constraints cache

    Example::

        >>> import textattack
        >>> import transformers

        >>> # Load model, tokenizer, and model_wrapper
        >>> model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")
        >>> tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
        >>> model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

        >>> # Construct our four components for `Attack`
        >>> from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
        >>> from textattack.constraints.semantics import WordEmbeddingDistance

        >>> goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)
        >>> constraints = [
        ...     RepeatModification(),
        ...     StopwordModification()
        ...     WordEmbeddingDistance(min_cos_sim=0.9)
        ... ]
        >>> transformation = WordSwapEmbedding(max_candidates=50)
        >>> search_method = GreedyWordSwapWIR(wir_method="delete")

        >>> # Construct the actual attack
        >>> attack = Attack(goal_function, constraints, transformation, search_method)

        >>> input_text = "I really enjoyed the new movie that came out last month."
        >>> label = 1 #Positive
        >>> attack_result = attack.attack(input_text, label)
    """

    def __init__(
        self,
        goal_function: GoalFunction,
        constraints: List[Union[Constraint, PreTransformationConstraint]],
        transformation: Transformation,
        search_method: SearchMethod,
        transformation_cache_size=2**15,
        constraint_cache_size=2**15,
    ):
        """Initialize an attack object.

        Attacks can be run multiple times.
        """
        assert isinstance(
            goal_function, GoalFunction
        ), f"`goal_function` must be of type `textattack.goal_functions.GoalFunction`, but got type `{type(goal_function)}`."
        assert isinstance(
            constraints, list
        ), "`constraints` must be a list of `textattack.constraints.Constraint` or `textattack.constraints.PreTransformationConstraint`."
        for c in constraints:
            assert isinstance(
                c, (Constraint, PreTransformationConstraint)
            ), "`constraints` must be a list of `textattack.constraints.Constraint` or `textattack.constraints.PreTransformationConstraint`."
        assert isinstance(
            transformation, Transformation
        ), f"`transformation` must be of type `textattack.transformations.Transformation`, but got type `{type(transformation)}`."
        assert isinstance(
            search_method, SearchMethod
        ), f"`search_method` must be of type `textattack.search_methods.SearchMethod`, but got type `{type(search_method)}`."

        self.goal_function = goal_function
        self.search_method = search_method
        self.transformation = transformation
        self.is_black_box = (
            getattr(transformation, "is_black_box", True) and search_method.is_black_box
        )

        if not self.search_method.check_transformation_compatibility(
            self.transformation
        ):
            raise ValueError(
                f"SearchMethod {self.search_method} incompatible with transformation {self.transformation}"
            )

        self.constraints = []
        self.pre_transformation_constraints = []
        for constraint in constraints:
            if isinstance(
                constraint,
                textattack.constraints.PreTransformationConstraint,
            ):
                self.pre_transformation_constraints.append(constraint)
            else:
                self.constraints.append(constraint)

        # Check if we can use transformation cache for our transformation.
        if not self.transformation.deterministic:
            self.use_transformation_cache = False
        elif isinstance(self.transformation, CompositeTransformation):
            self.use_transformation_cache = True
            for t in self.transformation.transformations:
                if not t.deterministic:
                    self.use_transformation_cache = False
                    break
        else:
            self.use_transformation_cache = True
        self.transformation_cache_size = transformation_cache_size
        self.transformation_cache = lru.LRU(transformation_cache_size)

        self.constraint_cache_size = constraint_cache_size
        self.constraints_cache = lru.LRU(constraint_cache_size)

        # Give search method access to functions for getting transformations and evaluating them
        self.search_method.get_transformations = self.get_transformations
        # Give search method access to self.goal_function for model query count, etc.
        self.search_method.goal_function = self.goal_function
        # The search method only needs access to the first argument. The second is only used
        # by the attack class when checking whether to skip the sample
        self.search_method.get_goal_results = self.goal_function.get_results

        self.search_method.filter_transformations = self.filter_transformations

    def clear_cache(self, recursive=True):
        self.constraints_cache.clear()
        if self.use_transformation_cache:
            self.transformation_cache.clear()
        if recursive:
            self.goal_function.clear_cache()
            for constraint in self.constraints:
                if hasattr(constraint, "clear_cache"):
                    constraint.clear_cache()

    def cpu_(self):
        """Move any `torch.nn.Module` models that are part of Attack to CPU."""
        visited = set()

        def to_cpu(obj):
            visited.add(id(obj))
            if isinstance(obj, torch.nn.Module):
                obj.cpu()
            elif isinstance(
                obj,
                (
                    Attack,
                    GoalFunction,
                    Transformation,
                    SearchMethod,
                    Constraint,
                    PreTransformationConstraint,
                    ModelWrapper,
                ),
            ):
                for key in obj.__dict__:
                    s_obj = obj.__dict__[key]
                    if id(s_obj) not in visited:
                        to_cpu(s_obj)
            elif isinstance(obj, (list, tuple)):
                for item in obj:
                    if id(item) not in visited and isinstance(
                        item, (Transformation, Constraint, PreTransformationConstraint)
                    ):
                        to_cpu(item)

        to_cpu(self)

    def cuda_(self):
        """Move any `torch.nn.Module` models that are part of Attack to GPU."""
        visited = set()

        def to_cuda(obj):
            visited.add(id(obj))
            if isinstance(obj, torch.nn.Module):
                obj.to(textattack.shared.utils.device)
            elif isinstance(
                obj,
                (
                    Attack,
                    GoalFunction,
                    Transformation,
                    SearchMethod,
                    Constraint,
                    PreTransformationConstraint,
                    ModelWrapper,
                ),
            ):
                for key in obj.__dict__:
                    s_obj = obj.__dict__[key]
                    if id(s_obj) not in visited:
                        to_cuda(s_obj)
            elif isinstance(obj, (list, tuple)):
                for item in obj:
                    if id(item) not in visited and isinstance(
                        item, (Transformation, Constraint, PreTransformationConstraint)
                    ):
                        to_cuda(item)

        to_cuda(self)

    def _get_transformations_uncached(self, current_text, original_text=None, **kwargs):
        """Applies ``self.transformation`` to ``text``, then filters the list
        of possible transformations through the applicable constraints.

        Args:
            current_text: The current ``AttackedText`` on which to perform the transformations.
            original_text: The original ``AttackedText`` from which the attack started.
        Returns:
            A filtered list of transformations where each transformation matches the constraints
        """
        transformed_texts = self.transformation(
            current_text,
            pre_transformation_constraints=self.pre_transformation_constraints,
            **kwargs,
        )

        return transformed_texts

    def get_transformations(self, current_text, original_text=None, **kwargs):
        """Applies ``self.transformation`` to ``text``, then filters the list
        of possible transformations through the applicable constraints.

        Args:
            current_text: The current ``AttackedText`` on which to perform the transformations.
            original_text: The original ``AttackedText`` from which the attack started.
        Returns:
            A filtered list of transformations where each transformation matches the constraints
        """
        if not self.transformation:
            raise RuntimeError(
                "Cannot call `get_transformations` without a transformation."
            )

        if self.use_transformation_cache:
            cache_key = tuple([current_text] + sorted(kwargs.items()))
            if utils.hashable(cache_key) and cache_key in self.transformation_cache:
                # promote transformed_text to the top of the LRU cache
                self.transformation_cache[cache_key] = self.transformation_cache[
                    cache_key
                ]
                transformed_texts = list(self.transformation_cache[cache_key])
            else:
                transformed_texts = self._get_transformations_uncached(
                    current_text, original_text, **kwargs
                )
                if utils.hashable(cache_key):
                    self.transformation_cache[cache_key] = tuple(transformed_texts)
        else:
            transformed_texts = self._get_transformations_uncached(
                current_text, original_text, **kwargs
            )

        return self.filter_transformations(
            transformed_texts, current_text, original_text
        )

    def _filter_transformations_uncached(
        self, transformed_texts, current_text, original_text=None
    ):
        """Filters a list of potential transformed texts based on
        ``self.constraints``

        Args:
            transformed_texts: A list of candidate transformed ``AttackedText`` to filter.
            current_text: The current ``AttackedText`` on which the transformation was applied.
            original_text: The original ``AttackedText`` from which the attack started.
        """
        filtered_texts = transformed_texts[:]
        for C in self.constraints:
            if len(filtered_texts) == 0:
                break
            if C.compare_against_original:
                if not original_text:
                    raise ValueError(
                        f"Missing `original_text` argument when constraint {type(C)} is set to compare against `original_text`"
                    )

                filtered_texts = C.call_many(filtered_texts, original_text)
            else:
                filtered_texts = C.call_many(filtered_texts, current_text)
        # Default to false for all original transformations.
        for original_transformed_text in transformed_texts:
            self.constraints_cache[(current_text, original_transformed_text)] = False
        # Set unfiltered transformations to True in the cache.
        for filtered_text in filtered_texts:
            self.constraints_cache[(current_text, filtered_text)] = True
        return filtered_texts

    def filter_transformations(
        self, transformed_texts, current_text, original_text=None
    ):
        """Filters a list of potential transformed texts based on
        ``self.constraints`` Utilizes an LRU cache to attempt to avoid
        recomputing common transformations.

        Args:
            transformed_texts: A list of candidate transformed ``AttackedText`` to filter.
            current_text: The current ``AttackedText`` on which the transformation was applied.
            original_text: The original ``AttackedText`` from which the attack started.
        """
        # Remove any occurences of current_text in transformed_texts
        transformed_texts = [
            t for t in transformed_texts if t.text != current_text.text
        ]
        # Populate cache with transformed_texts
        uncached_texts = []
        filtered_texts = []
        for transformed_text in transformed_texts:
            if (current_text, transformed_text) not in self.constraints_cache:
                uncached_texts.append(transformed_text)
            else:
                # promote transformed_text to the top of the LRU cache
                self.constraints_cache[
                    (current_text, transformed_text)
                ] = self.constraints_cache[(current_text, transformed_text)]
                if self.constraints_cache[(current_text, transformed_text)]:
                    filtered_texts.append(transformed_text)
        filtered_texts += self._filter_transformations_uncached(
            uncached_texts, current_text, original_text=original_text
        )
        # Sort transformations to ensure order is preserved between runs
        filtered_texts.sort(key=lambda t: t.text)
        return filtered_texts

    def _attack(self, initial_result):
        """Calls the ``SearchMethod`` to perturb the ``AttackedText`` stored in
        ``initial_result``.

        Args:
            initial_result: The initial ``GoalFunctionResult`` from which to perturb.

        Returns:
            A ``SuccessfulAttackResult``, ``FailedAttackResult``,
                or ``MaximizedAttackResult``.
        """
        final_result = self.search_method(initial_result)
        self.clear_cache()
        if final_result.goal_status == GoalFunctionResultStatus.SUCCEEDED:
            result = SuccessfulAttackResult(
                initial_result,
                final_result,
            )
            print(initial_result, final_result)
        elif final_result.goal_status == GoalFunctionResultStatus.SEARCHING:
            result = FailedAttackResult(
                initial_result,
                final_result,
            )
        elif final_result.goal_status == GoalFunctionResultStatus.MAXIMIZING:
            result = MaximizedAttackResult(
                initial_result,
                final_result,
            )
        else:
            raise ValueError(f"Unrecognized goal status {final_result.goal_status}")
        return result

    def attack(self, example, ground_truth_output):
        """Attack a single example.

        Args:
            example (:obj:`str`, :obj:`OrderedDict[str, str]` or :class:`~textattack.shared.AttackedText`):
                Example to attack. It can be a single string or an `OrderedDict` where
                keys represent the input fields (e.g. "premise", "hypothesis") and the values are the actual input textx.
                Also accepts :class:`~textattack.shared.AttackedText` that wraps around the input.
            ground_truth_output(:obj:`int`, :obj:`float` or :obj:`str`):
                Ground truth output of `example`.
                For classification tasks, it should be an integer representing the ground truth label.
                For regression tasks (e.g. STS), it should be the target value.
                For seq2seq tasks (e.g. translation), it should be the target string.
        Returns:
            :class:`~textattack.attack_results.AttackResult` that represents the result of the attack.
        """
        assert isinstance(
            example, (str, OrderedDict, AttackedText)
        ), "`example` must either be `str`, `collections.OrderedDict`, `textattack.shared.AttackedText`."
        if isinstance(example, (str, OrderedDict)):
            example = AttackedText(example)

        assert isinstance(
            ground_truth_output, (int, str, float)
        ), "`ground_truth_output` must either be `str` or `int`."
        goal_function_result, _ = self.goal_function.init_attack_example(
            example, ground_truth_output
        )
        if goal_function_result.goal_status == GoalFunctionResultStatus.SKIPPED:
            return SkippedAttackResult(goal_function_result)
        else:
            result = self._attack(goal_function_result)
            return result

    def __repr__(self):
        """Prints attack parameters in a human-readable string.

        Inspired by the readability of printing PyTorch nn.Modules:
        https://github.com/pytorch/pytorch/blob/master/torch/nn/modules/module.py
        """
        main_str = "Attack" + "("
        lines = []

        lines.append(utils.add_indent(f"(search_method): {self.search_method}", 2))
        # self.goal_function
        lines.append(utils.add_indent(f"(goal_function):  {self.goal_function}", 2))
        # self.transformation
        lines.append(utils.add_indent(f"(transformation):  {self.transformation}", 2))
        # self.constraints
        constraints_lines = []
        constraints = self.constraints + self.pre_transformation_constraints
        if len(constraints):
            for i, constraint in enumerate(constraints):
                constraints_lines.append(utils.add_indent(f"({i}): {constraint}", 2))
            constraints_str = utils.add_indent("\n" + "\n".join(constraints_lines), 2)
        else:
            constraints_str = "None"
        lines.append(utils.add_indent(f"(constraints): {constraints_str}", 2))
        # self.is_black_box
        lines.append(utils.add_indent(f"(is_black_box):  {self.is_black_box}", 2))
        main_str += "\n  " + "\n  ".join(lines) + "\n"
        main_str += ")"
        return main_str

    def __getstate__(self):
        state = self.__dict__.copy()
        state["transformation_cache"] = None
        state["constraints_cache"] = None
        return state

    def __setstate__(self, state):
        self.__dict__ = state
        self.transformation_cache = lru.LRU(self.transformation_cache_size)
        self.constraints_cache = lru.LRU(self.constraint_cache_size)

    __str__ = __repr__


## Recipe

In [ ]:
tokenizer.model_max_length

512

In [ ]:
class HuggingFaceWrapper(PyTorchModelWrapper):
    """Loads a HuggingFace ``transformers`` model and tokenizer."""

    def __init__(self, model, tokenizer):
        # assert isinstance(
        #     model, transformers.PreTrainedModel
        # ), f"`model` must be of type `transformers.PreTrainedModel`, but got type {type(model)}."
        # assert isinstance(
        #     tokenizer,
        #     (transformers.PreTrainedTokenizer, transformers.PreTrainedTokenizerFast),
        # ), f"`tokenizer` must of type `transformers.PreTrainedTokenizer` or `transformers.PreTrainedTokenizerFast`, but got type {type(tokenizer)}."

        self.model = model
        self.tokenizer = tokenizer

    def __call__(self, text_input_list):
        """Passes inputs to HuggingFace models as keyword arguments.

        (Regular PyTorch ``nn.Module`` models typically take inputs as
        positional arguments.)
        """
        # Default max length is set to be int(1e30), so we force 512 to enable batching.
        max_length = (
            512
            if self.tokenizer.model_max_length == int(1e30)
            else self.tokenizer.model_max_length
        )
        inputs_dict = self.tokenizer(
            text_input_list,
            add_special_tokens=True,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        )
        
        model_device = next(self.model.parameters()).device
        inputs_dict.to(model_device)

        with torch.no_grad():
            self.model.eval()
            outputs = self.model(**inputs_dict)

        if isinstance(outputs[0], str):
            # HuggingFace sequence-to-sequence models return a list of
            # string predictions as output. In this case, return the full
            # list of outputs.
            return outputs
        else:
            # HuggingFace classification models return a tuple as output
            # where the first item in the tuple corresponds to the list of
            # scores for each input.
            # print(outputs[0])
            return outputs[0]

    def get_grad(self, text_input):
        """Get gradient of loss with respect to input tokens.

        Args:
            text_input (str): input string
        Returns:
            Dict of ids, tokens, and gradient as numpy array.
        """
        if isinstance(self.model, textattack.models.helpers.T5ForTextToText):
            raise NotImplementedError(
                "`get_grads` for T5FotTextToText has not been implemented yet."
            )

        self.model.train()
        embedding_layer = self.model.get_input_embeddings()
        original_state = embedding_layer.weight.requires_grad
        embedding_layer.weight.requires_grad = True

        emb_grads = []

        def grad_hook(module, grad_in, grad_out):
            emb_grads.append(grad_out[0])

        emb_hook = embedding_layer.register_backward_hook(grad_hook)

        self.model.zero_grad()
        model_device = next(self.model.parameters()).device
        input_dict = self.tokenizer(
            [text_input],
            add_special_tokens=True,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
        )
        input_dict.to(model_device)
        predictions = self.model(**input_dict).logits

        try:
            labels = predictions.argmax(dim=1)
            loss = self.model(**input_dict, labels=labels.float())[0]
        except TypeError:
            raise TypeError(
                f"{type(self.model)} class does not take in `labels` to calculate loss. "
                "One cause for this might be if you instantiatedyour model using `transformer.AutoModel` "
                "(instead of `transformers.AutoModelForSequenceClassification`)."
            )

        loss.backward()

        # grad w.r.t to word embeddings
        grad = emb_grads[0][0].cpu().numpy()

        embedding_layer.weight.requires_grad = original_state
        emb_hook.remove()
        self.model.eval()

        output = {"ids": input_dict["input_ids"], "gradient": grad}

        return output


    def _tokenize(self, inputs):
        """Helper method that for `tokenize`
        Args:
            inputs (list[str]): list of input strings
        Returns:
            tokens (list[list[str]]): List of list of tokens as strings
        """
        return [
            self.tokenizer.convert_ids_to_tokens(
                self.tokenizer([x], truncation=True)["input_ids"][0]
            )
            for x in inputs
        ]


In [ ]:
def create_attack(model_wrapper, attack_name='A2T'):
    
    if attack_name == 'BAE'  :
        transformation = WordSwapMaskedLM(method="bae", max_candidates=50, min_confidence=0.0)

        constraints = [RepeatModification(), StopwordModification()]

        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))
        use_constraint = UniversalSentenceEncoder(
            threshold=0.4,
            metric="cosine",
            compare_against_original=True,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)
        goal_function = TargetedRegression(model_wrapper)

        search_method = GreedyWordSwapWIR(wir_method="delete")
        print('BAE attack is running.....')
        return Attack(goal_function, constraints, transformation, search_method)
            
    elif attack_name == 'A2T' :
        constraints = [RepeatModification(), StopwordModification()]
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        constraints.append(PartOfSpeech(allow_verb_noun_swap=False))
        constraints.append(MaxModificationRate(max_rate=0.1, min_threshold=4))
        sent_encoder = BERT(
            model_name="stsb-distilbert-base", threshold=0.60, metric="cosine"
        )
        constraints.append(sent_encoder)

        transformation = transformation = WordSwapMaskedLM(
            method="bae", max_candidates=20, min_confidence=0.0, batch_size=32
        )

        goal_function = TargetedRegression(model_wrapper, model_batch_size=32)

        search_method = GreedyWordSwapWIR(wir_method="gradient")
        print('A2T attack is running.....')
        return Attack(goal_function, constraints, transformation, search_method)

    elif attack_name == 'Text-Fooler':
        transformation = WordSwapEmbedding(max_candidates=50)

        stopwords = set(
            ["a", "about", "above", "across", "after", "afterwards", "again", "against", "ain", "all", "almost", "alone", "along", "already", "also", "although", "am", "among", "amongst", "an", "and", "another", "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "aren", "aren't", "around", "as", "at", "back", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "both", "but", "by", "can", "cannot", "could", "couldn", "couldn't", "d", "didn", "didn't", "doesn", "doesn't", "don", "don't", "down", "due", "during", "either", "else", "elsewhere", "empty", "enough", "even", "ever", "everyone", "everything", "everywhere", "except", "first", "for", "former", "formerly", "from", "hadn", "hadn't", "hasn", "hasn't", "haven", "haven't", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "i", "if", "in", "indeed", "into", "is", "isn", "isn't", "it", "it's", "its", "itself", "just", "latter", "latterly", "least", "ll", "may", "me", "meanwhile", "mightn", "mightn't", "mine", "more", "moreover", "most", "mostly", "must", "mustn", "mustn't", "my", "myself", "namely", "needn", "needn't", "neither", "never", "nevertheless", "next", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "o", "of", "off", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "per", "please", "s", "same", "shan", "shan't", "she", "she's", "should've", "shouldn", "shouldn't", "somehow", "something", "sometime", "somewhere", "such", "t", "than", "that", "that'll", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "this", "those", "through", "throughout", "thru", "thus", "to", "too", "toward", "towards", "under", "unless", "until", "up", "upon", "used", "ve", "was", "wasn", "wasn't", "we", "were", "weren", "weren't", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "with", "within", "without", "won", "won't", "would", "wouldn", "wouldn't", "y", "yet", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"]
        )

        constraints = [RepeatModification(), StopwordModification(stopwords=stopwords)]

        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        constraints.append(WordEmbeddingDistance(min_cos_sim=0.2))
        constraints.append(PartOfSpeech(allow_verb_noun_swap=True))

        use_constraint = UniversalSentenceEncoder(
            threshold=0.40,
            metric="angular",
            compare_against_original=False,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)

        goal_function = TargetedRegression(model_wrapper)

        search_method = GreedyWordSwapWIR(wir_method="delete")
        print('Text-Fooler attack is running.....')
        return Attack(goal_function, constraints, transformation, search_method)

    elif attack_name == 'CLARE':
        shared_masked_lm = transformers.AutoModelForCausalLM.from_pretrained("distilroberta-base")
        shared_tokenizer = transformers.AutoTokenizer.from_pretrained("distilroberta-base")
        transformation = CompositeTransformation(
            [
                WordSwapMaskedLM(
                    method="bae",
                    masked_language_model=shared_masked_lm,
                    tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=5e-4,
                ),
                WordInsertionMaskedLM(
                    masked_language_model=shared_masked_lm,
                    tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=0.0,
                ),
                WordMergeMaskedLM(
                    masked_language_model=shared_masked_lm,
                    tokenizer=shared_tokenizer,
                    max_candidates=50,
                    min_confidence=5e-3,
                ),
            ]
        )

        constraints = [RepeatModification(), StopwordModification()]

        use_constraint = UniversalSentenceEncoder(
            threshold=0.7,
            metric="cosine",
            compare_against_original=True,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(use_constraint)

        goal_function = TargetedRegression(model_wrapper)

        search_method = GreedySearch()
        print('CLARE attack is running.....')
        return Attack(goal_function, constraints, transformation, search_method)

    elif attack_name == "Bert-attack":
        transformation = WordSwapMaskedLM(method="bert-attack", max_candidates=48)
        
        constraints = [RepeatModification(), StopwordModification()]
        constraints.append(MaxWordsPerturbed(max_percent=0.4))
        use_constraint = UniversalSentenceEncoder(
            threshold=0.2,
            metric="cosine",
            compare_against_original=True,
            window_size=None,
        )
        constraints.append(use_constraint)
        
        goal_function = TargetedRegression(model_wrapper)
        
        search_method = GreedyWordSwapWIR(wir_method="unk")
        print('Bert-attack attack is running.....')
        return Attack(goal_function, constraints, transformation, search_method)
    
    elif attack_name == "A2T-Hybrid":
        constraints = [RepeatModification(), StopwordModification()]
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        constraints.append(PartOfSpeech(allow_verb_noun_swap=False))
        constraints.append(MaxModificationRate(max_rate=0.1, min_threshold=4))
        sent_encoder = UniversalSentenceEncoder(
            threshold=0.4,
            metric="cosine",
            compare_against_original=True,
            window_size=20,
            skip_text_shorter_than_window=True,
        )
        constraints.append(sent_encoder)
        shared_masked_lm = transformers.AutoModelForCausalLM.from_pretrained("distilroberta-base")
        shared_tokenizer = transformers.AutoTokenizer.from_pretrained("distilroberta-base")

        transformation = WordSwapMaskedLM(
                    method="bae",
                    masked_language_model=shared_masked_lm,
                    tokenizer=shared_tokenizer,
                    max_candidates=20,
                    min_confidence=5e-4,
                )

        goal_function = TargetedRegression(model_wrapper, model_batch_size=32)

        search_method = GreedyWordSwapWIR(wir_method="gradient")
        print('A2T-Hybrid1 attack is running.....')
        return Attack(goal_function, constraints, transformation, search_method)

    elif attack_name == "A2T-Hybrid_v1":
        constraints = [RepeatModification(), StopwordModification()]
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        constraints.append(PartOfSpeech(allow_verb_noun_swap=False))
        constraints.append(MaxModificationRate(max_rate=0.15, min_threshold=4))
        sent_encoder = UniversalSentenceEncoder(
            threshold=0.5,
            metric="cosine",
            compare_against_original=True,
            window_size=15,
            skip_text_shorter_than_window=True,
        )
        constraints.append(sent_encoder)
        shared_masked_lm = transformers.AutoModelForCausalLM.from_pretrained("distilroberta-base")
        shared_tokenizer = transformers.AutoTokenizer.from_pretrained("distilroberta-base")

        transformation = WordSwapMaskedLM(
                    method="bae",
                    masked_language_model=shared_masked_lm,
                    tokenizer=shared_tokenizer,
                    max_candidates=20,
                    min_confidence=5e-4,
                )

        goal_function = TargetedRegression(model_wrapper, model_batch_size=32)

        search_method = GreedyWordSwapWIR(wir_method="unk")
        print('A2T-Hybrid_v1 attack is running.....')
        return Attack(goal_function, constraints, transformation, search_method)

    elif attack_name == "A2T-Hybrid_v2":
        constraints = [RepeatModification(), StopwordModification()]
        input_column_modification = InputColumnModification(
            ["premise", "hypothesis"], {"premise"}
        )
        constraints.append(input_column_modification)
        constraints.append(PartOfSpeech(allow_verb_noun_swap=False))
        constraints.append(MaxModificationRate(max_rate=0.1, min_threshold=4))
        sent_encoder = BERT(
            model_name="stsb-distilbert-base", threshold=0.40, metric="cosine"
        )
        constraints.append(sent_encoder)
        shared_masked_lm = transformers.AutoModelForCausalLM.from_pretrained("distilroberta-base")
        shared_tokenizer = transformers.AutoTokenizer.from_pretrained("distilroberta-base")
        transformation = WordSwapMaskedLM(
                    method="bae",
                    masked_language_model=shared_masked_lm,
                    tokenizer=shared_tokenizer,
                    max_candidates=20,
                    min_confidence=5e-4,
                )
        goal_function = TargetedRegression(model_wrapper, model_batch_size=32)
        search_method = GreedyWordSwapWIR(wir_method="gradient")
        print('A2T-Hybrid2 attack is running.....')
        return Attack(goal_function, constraints, transformation, search_method)

    else:
        raise Exception("Only, 'BAE', 'A2T','Bert-attack', 'CLARE' and 'Text-Fooler' attack is implemented here. The name is case sensitive :)")

## Run

In [ ]:
model_wrapper = HuggingFaceWrapper(model, tokenizer)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
model_wrapper.to(device)

attack = create_attack(model_wrapper, attack_name='A2T')

loading configuration file /root/.cache/torch/sentence_transformers/sentence-transformers_stsb-distilbert-base/config.json
Model config DistilBertConfig {
  "_name_or_path": "old_models/stsb-distilbert-base/0_Transformer",
  "activation": "gelu",
  "architectures": [
    "DistilBertModel"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.12.2",
  "vocab_size": 30522
}

loading weights file /root/.cache/torch/sentence_transformers/sentence-transformers_stsb-distilbert-base/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertModel.

All the weights of DistilBertModel were initialized from the model checkpoint at /root/.cache/torch/sentence

A2T attack is running.....


In [ ]:
result = attack.attack(response, score)
print(score)
print(result.__str__(color_method='ansi'))
# print(model_wrapper(response))
# model_wrapper(result.perturbed_text())

In [ ]:
prompt = 'You are helping an event planner with an important event that involves coordinating several teams working together to setup the hired equipment, deliver the event, and tidy up afterwards. To ensure the event runs smoothly, you have been asked to prepare for anything that could go wrong. What types of problems, both internal and external to your team, could come up that would affect how well the event goes? List as many specific factors as you can'
prompt

'You are helping an event planner with an important event that involves coordinating several teams working together to setup the hired equipment, deliver the event, and tidy up afterwards. To ensure the event runs smoothly, you have been asked to prepare for anything that could go wrong. What types of problems, both internal and external to your team, could come up that would affect how well the event goes? List as many specific factors as you can'

In [ ]:
attack_df = pd.DataFrame()
df = test_set[['transcript', 'content_rating']][test_set[ 'content_rating']<3]
for idx, row in tqdm(df.iterrows()):
    result = attack.attack(row['transcript'], row['content_rating'])
    attack_df = attack_df.append({'transcript': row['transcript']
                                  , 'content_rating':row['content_rating']
                                  , 'Adversarial':result.perturbed_text()
                                  , 'dump' : result.str_lines()[0]}, ignore_index=True)
    result.original_text(), result.perturbed_text()
    print(row['content_rating'])
    print(result.__str__(color_method='ansi'))

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:1033: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
1it [00:11, 11.64s/it]

<textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f57fe6a8450> <textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f57ec4df1d0>
1.0
0 (103%) --> 0 (251%)

We need to focus firstly on delegating tasks and priorities. We need someone to focus on. Does the equipment arrive? Is the hall empty? Are people arriving early? And where do we stage them? If our room is not available? Secondly, we need to plan for. People's problems if there's a situation or an escalation, how do we remove a person from the situation? How do we deescalate the situation? We need to work with the management of the venue to find out exactly what they offer and if there is any restrictions or unavailable items. How do we prepare for that? Or how do we handle that if, say, their room is not available after the last party? What do we do? Who do we talk to? We need to have those resources on h

2it [00:20, 10.20s/it]

1.0
0 (99%) --> [FAILED]

If I were coordinating an event that was very complex, involves many team members, I would prepare in many different ways to make sure everything ran smoothly. I would have extras of every material thing I would need, so depending on what the event is, but let's say it's a banquet, extra linens, extra candles, extra drinks, I would have a team meeting at the beginning of the event at the beginning of call time for the event I would speak to each team member individually. Make sure that they're on board and that they feel a part of the mission. That would make sure to check in throughout the entire shift. That everyone's doing what they should be doing, but also that they feel appreciated and that we care about them being here. I would prepare by. Assuring that everything that needs to be there will be there if it's food. If it's a guest list, it depends, but I would just make sure many times. 


3it [00:27,  8.60s/it]

...

<textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f57fe9a9a50> <textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f57ec134310>
1.0
0 (100%) --> 0 (253%)

You are helping an event planner with an important event that involves coordinating several teams working together to set up the hired equipment, deliver the event and tidy up afterwards to ensure the event goes smoothly. You have been asked to prepare for anything that could go wrong. What types of problems, both internal and external to your team could you come up with that could affect how well the event goes? List has many specific factors. If you as you can well, I would go with the checklist and a couple of different accountability partners just to make sure that all of the different. All of the different preferred, I'm sorry all of the different things that we need to have happen get done. 

You are

35it [05:05,  7.76s/it]

1.0
0 (102%) --> [FAILED]

OK, so a lot can go wrong with event planning. You gotta make sure you hire the right people. The best people you can find, but you've got to assume that some people aren't going to show up. You've got a book, the venue. Let everyone know when they're supposed to be there. Let everyone know what their job is. You're definitely. Gonna have people who don't want to clean up afterwards, but you're gonna have to assign all these jobs beforehand. You gotta make sure you have the equipment on time. You gotta make sure you have food. All the teams set up all the room setup. Wi-Fi for any work you gotta. You know, just make sure everyone has everything they need, but assume that some people aren't gonna show up. Some things are gonna go wrong, you've gotta just. Plan for you know, plan for the best, but be prepared for the worst. 


36it [05:06,  8.52s/it]

2.0
0 (189%) --> [FAILED]

The potential problems you could have is that supplies didn't turn up the equipment that they supply is unusable. How to save someone non factors like the pandemic as such could come into effect. 


In [ ]:
attack_df.to_csv('/content/A2T_attack_df_763_+1.5.csv')

In [ ]:
attack = create_attack(model_wrapper, attack_name='A2T-Hybrid_v1')

df = test_set[['transcript', 'content_rating']][test_set[ 'content_rating']<3]
for idx, row in tqdm(df.iterrows()):
    result = attack.attack(row['transcript'], row['content_rating'])
    attack_df = attack_df.append({'transcript': row['transcript']
                                  , 'content_rating':row['content_rating']
                                  , 'Adversarial':result.perturbed_text()
                                  , 'dump' : result.str_lines()[0]}, ignore_index=True)
    result.original_text(), result.perturbed_text()
    print(row['content_rating'])
    print(result.__str__(color_method='ansi'))

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/distilroberta-base/resolve/main/pytorch_mode

A2T-Hybrid_v1 attack is running.....


1it [00:18, 18.43s/it]

1.0
0 (103%) --> [FAILED]

We need to focus firstly on delegating tasks and priorities. We need someone to focus on. Does the equipment arrive? Is the hall empty? Are people arriving early? And where do we stage them? If our room is not available? Secondly, we need to plan for. People's problems if there's a situation or an escalation, how do we remove a person from the situation? How do we deescalate the situation? We need to work with the management of the venue to find out exactly what they offer and if there is any restrictions or unavailable items. How do we prepare for that? Or how do we handle that if, say, their room is not available after the last party? What do we do? Who do we talk to? We need to have those resources on hand. The other thing we need to prioritize is the specific expectations and requests of the facilitators. What is it you really need for us? What is the number one focus on what we need? Who do we follow up with afterward? Both for billing and satisfaction, 

2it [00:28, 13.80s/it]

...

2.0
0 (193%) --> [FAILED]

So I think the person foremost in terms of actually setting up the equipment. One of the most important things is sort of ensuring that all the well. Whichever teams or staff members are involved in using the equipment or. Adequately trained in actually operating the equipment and sort of if any issues come about when attempting to use the equipment, they're able to troubleshoot or go to somebody that would be able to help them. Basically fix any issues promptly. Then yeah, also, everyone just needs to know what they're doing in terms of delivering the event so people have to stick to timelines, guidelines, deliver the information appropriately, and keep the audience engaged and in terms of tidying up, this also needs to be done in a prompt fashion, ensuring that the equipment is not damaged and is transported from A to B in an adequate timeframe. 


30it [09:07, 11.46s/it]

0.0
0 (-4%) --> [FAILED]

Basically I've been ahead. To prepare anything. Back up, go running. Why the change are working together and basically you make sure that you can use Tyler. Cleaning Charlie around there's no obstruction, use it. While and whilst on the floor so anybody could select a file with no wires. Life wise I'm going around. Make sure everything's in order in place. And basically basically concerned. 


31it [21:56, 238.89s/it]

1.0
0 (103%) --> [FAILED]

So look through the event, set up delivery and tidy up. The first thing to do is get the teams together. Make sure everyone's there and they all know what they're doing and setup is important and need to get the equipment together and everyone needs to know what they're doing with it. So sufficient training is very important. Depending on the equipment. The man has to be there on time, has to be distributed to the correct locations. And the teams need to know what they're doing with their. If there's any equipment, light broken or waiting on it. The day could be quite delayed. We've got to deliver the. Event as well. As far as delivery goes, it just rains probably a bit more standing around, making sure everything carries on working and the teams and individuals know what to do with the people and tidying up people hate this and always want to go home. It could be not enough bins. It could be that stuff can't be packed away. 


In [ ]:
attack = create_attack(model_wrapper, attack_name='BAE')

df = test_set[['transcript', 'content_rating']][test_set[ 'content_rating']<2][:5]
for idx, row in tqdm(df.iterrows()):
    result = attack.attack(row['transcript'], row['content_rating'])
    # attack_df = attack_df.append({'transcript': row['transcript']
    #                               , 'content_rating':row['content_rating']
    #                               # , 'domain_name':row['domain_name']
    #                               , 'Adversarial':result.perturbed_text()
    #                               # ,'intent_name': row['intent_name']
    #                               , 'dump' : result.str_lines()[0]}, ignore_index=True)
    # result.original_text(), result.perturbed_text()
    print(row['content_rating'])
    print(result.__str__(color_method='ansi'))

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache 

BAE attack is running.....


1it [00:12, 12.15s/it]

<textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f133c417550> <textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f13464e9450>
1.0
0 (115%) --> 0 (254%)

I think with this type of task it's always important to. So delegate. What do you want? What do you want people to do? So in your teams, maybe have somebody that's like the head person. And there are people that are underneath them so. If is that the work and needs help, they can go to maybe the manager and that manager can come to you. So it's like a hierarchy and that usually helps things go smoothly and also right down the tasks that you want each. Each department to do and then they can always tick that off. But As for external to make sure that you're prepared for every eventuality. Maybe hiring extra staff just in case. I have staff on standby in case there's some people don't turn up. 

I think with

2it [00:18,  8.69s/it]

<textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f133c417d50> <textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f13465aa050>
1.0
0 (112%) --> 0 (266%)

If the event is planned for outdoors and there should always be something on the side just in case to be able to go indoors. In case of the weather not cooperating for that, and I would say always had extra event assistance for set up for take down for cleanup. In any event that someone can't show up that day to help out. Always have an extra on hand is a great idea in any type of event that you're planning. Check, recheck all of your lists. Make sure you have everything. Call and verify that all the deliveries are scheduled for certain time. Verify the time and follow up with that. If you have to. The date of the event or day before the event just to make sure that everything is covered. And you have back

3it [00:24,  7.68s/it]

<textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f133d7a50d0> <textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f134668bcd0>
1.0
0 (124%) --> 0 (250%)

It would have to be a thorough risk assessment for like health and safety and stuff like that. I would when it comes to setting things up. Umm? Tidy up after her. Set the height equipment. What I would do is I would. Not being sexist, I would probably get guys who are well built to build physical stuff rather than. Guys who are not as well built or women just to avoid any. Need to fill out, you know, like an accident form or anything like that. That's how I'd put different teams together based on their physical capability. 

It would have to be a financial risk involved for like health and safety and stuff like that. I would when it comes to setting things up. Umm? working up after her. forget the height e

5it [00:31,  6.30s/it]

<textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f1346425f90> <textattack.goal_function_results.classification_goal_function_result.ClassificationGoalFunctionResult object at 0x7f1346bd47d0>
1.0
0 (145%) --> 0 (250%)

So yes, I would split it into UM. Into three different sections, so I'd have anything to do with the post event, so that will be organized and suppliers organized, supplies how they would get there, any sort of access for trade routes, trade suppliers, rather, how they would get to the when we get to the site and anything on initial setup. So if you need to security. It just depends how large slot event is really UM, and then I'd have a middle section while the the event was in place for how the teams would work together during the event. How they would cooperate work together if there was any crossover between teams that reached team had a specific skill set that they would use source that to how they wou

In [ ]:
response, score = test_set[['transcript', 'content_rating']].values[0]
print(response)
print(score)
print(model_wrapper(response))
# response = 'The first thing that came into my mind was like this is an outdoor or indoor event and whether the weather may have been influence on possible problems, so that would certainly have to be taken into account. Umm? And what kind of equipment are we using for the event and? What was the risk of not receiving the equipment in time for the event? Who is what type of people are going to attend the event? Do we need to make any specific? Arrangements for their needs. And yeah, these were the main things that came into mind.'
model_wrapper(result.perturbed_text())

The types of problems and that could come up that would affect how well the event goes or for example and the hired equipment doesn't arrive. Or perhaps it arrives with parts missing. And. The setup of that equipment. Perhaps there is a lack of technical knowledge within the team that is setting that up, or there's a problem. Perhaps the equipment isn't what was expected in the delivery of the event because we're coordinating several teams who are working together. There could be problems at coordinating those and an agreement between who does what and who is responsible for what. And similarly, that could affect the tidy up afterwards. And there could also be problems with the collection of the hired equipment at the end. Perhaps then you policies around recycling and rubbish disposal that hadn't been considered. And so, yeah, a plethora of problems that could come up and that would affect the event. 
3.0
tensor([[3.0061]], device='cuda:0')


AttributeError: ignored

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/SNGP_BERT_Pytorch/dataset/clinc_oos/test.csv')
test_df.head()

,text,intent,domain,intent_name,domain_name
0,how would you say fly in italian,84,5,translate,travel
1,what 's the spanish word for pasta,84,5,translate,travel
2,how would they say butter in zambia,84,5,translate,travel
3,how do you say fast in spanish,84,5,translate,travel
4,what 's the word for trees in norway,84,5,translate,travel


In [ ]:
df = response, score = test_set[['transcript', 'content_rating']]
df.reset_index(inplace=True, drop=True)
df.head()

,transcript,content_rating
0,So having worked in the past on some different...,1.0
1,Things that could go wrong. Stuff gets sick. P...,4.0
2,"OK, the things that could go wrong. First of a...",5.0
3,The specific times of programs that we could e...,3.0
4,There are so many factors that could come up a...,3.0


In [ ]:
# model_wrapper(['how are you'])

In [ ]:
# df = test_df
# df.reset_index(inplace=True, drop=True)

attack_df = pd.DataFrame()
# attack = Attack(goal_function, constraints, transformation, search_method)
for idx, row in tqdm(df.iterrows()):
    result = attack.attack(row['transcript'], int(row['content_rating']))
    attack_df = attack_df.append({'transcript': row['transcript']
                                  , 'content_rating':row['content_rating']
                                  # , 'domain_name':row['domain_name']
                                  , 'Adversarial':result.perturbed_text()
                                  # ,'intent_name': row['intent_name']
                                  , 'dump' : result.str_lines()[0]}, ignore_index=True)
    result.original_text(), result.perturbed_text()
    print(result)
    

4it [00:00, 31.99it/s]

0 (100%) --> [SKIPPED]

So having worked in the past on some different types of events in a variety of venues I can see there's a number of issues that can go wrong, particularly when there's higher equipment. One of the main things that is most important to mitigate against some of these issues is to ensure that there's a lot of forward planning to ensure that any kind of risks are managed and are mitigated against. So, for example, ensuring that the planning process that you make sure that all of the technical specifications. Are really specific and clear to avoid any incompatibility issues. That's a big issue. Another one is is just staff not having an awareness, so just making sure that staff are appropriately trained for the equipment that may be in place, making sure that you've got a full inventory log. Make sure that nothing doesn't. Equipment doesn't turn up or similar that you know that you know that you're getting what you need. Also, making sure that your clients or your pr

13it [00:00, 37.29it/s]

0 (100%) --> [SKIPPED]

So in terms of all the things that could go wrong, we could think about our team members. Of course, everyone is briefed and has been allocated a role on the event day, but it could happen that somebody is sick or absent. So we need to have extra people available in terms of the higher equipment we need to make sure that arrives on time that the right people are there to put those pieces of equipment together and to monitor it throughout the event. Otherwise, if something breaks down, we will have a problem in terms of the person and the people delivering the presentations at the event they could be delayed and there could be some travel issues. Train or underground cancellations. Indeed, the audience could be affected by the same thing, so we need to consider. Whether it might be cancelled last minute? Early in terms of tidying up, we need to make sure we have a team of people who are available to recycle everything and make sure everything is done correctly an

25it [00:00, 45.22it/s]

...

0 (100%) --> [SKIPPED]

Right, I would say there are other things that could go wrong with this. Firstly, people who were meant to turn up on site on the day either don't turn up at all or they're late. Additionally, equipment that you plan to come along will be delivered, doesn't turn up, or if it turns up it doesn't work. Delays in things arriving delays in people arriving on site systems might not work either, such as Internet, electricity, other things such as that. In other words, a failure of equipment, people and systems, or delays in any of the which. 
0 (100%) --> [SKIPPED]

So I would make sure to coordinate in a way where you know I'll try to anticipate you know several different types of problems. I know that there would be both internal and external problems that would come up with an event. You know. I imagine there's stuff like weather that could show you know weather issues that could make an event kind of flop. I know that externally people could be late, both my own t

87it [00:20,  1.11s/it]

0 (100%) --> [FAILED]

So I would look at the various companies websites. To ensure that they have the specification required for our requirements. Set up some sort of. Excel spreadsheets. Giving the various key features of the different manufacturers equipment and ranking them by price. 
0 (100%) --> [SKIPPED]

OK, so things could go wrong. The people working together might not like each other, but I could get to an argument and there could be like a good leader. A person might not show up on time to the event. People might not have the right cell phone numbers. They might not understand what time they need to be there. They might screw up the date. They might not understand who's obligation is to do different types of jobs. People might not understand kind of what the purpose of the event is. People might not understand whose job is to clean up things. There might be other kind of messes that people don't anticipate. Break something or someone spill something. Or there could be bad w

100it [00:20,  4.76it/s]

0 (100%) --> [SKIPPED]

I mean, this is a pretty vague question really, isn't it? What's the event to start with? So. I mean, you could have you know you've got hiring equipment. There could be a breakdown supply chain problem accident on the motorway, you don't get any of your equipment. Perhaps it's been a Miss order, some equipments missing, you know to account for that, find a way to get an alternative. Suppose it delivery the event well. If one of your speakers doesn't turn up somebody's missing or. You know, not enough tables for guests that sort of thing, making sure that you've got enough and everything is organized correctly, tidying up afterwards. I mean, there's not really that much can go wrong there, just making sure you've got enough equipment and stuff to get rid of everything. Making sure everyone is prepared, I suppose is the highest thing to think about, but I mean without knowing the event that's a very difficult question to answer. 
0 (100%) --> [SKIPPED]

Possible 